<a href="https://colab.research.google.com/github/HamzaAhmed78629/Python-Coding-Projects/blob/main/AES-RSA-DigitalSignature-Encryption%2BDecryptionAlgorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install cryptography

In [ ]:
pip install pycryptodome

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 17.0 MB/s eta 0:00:00


In [ ]:
pip install eciespy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 47.9 MB/s eta 0:00:00


In [ ]:
pip install bcrypt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.9/698.9 kB 7.9 MB/s eta 0:00:00


In [ ]:
# Importinging the necessary modules from the Crypto library
from Crypto.Cipher import AES, PKCS1_OAEP
from Crypto.Random import get_random_bytes
from Crypto.Protocol.KDF import PBKDF2
from Crypto.Hash import SHA256
from Crypto.Util.Padding import pad, unpad
from Crypto.PublicKey import RSA
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.backends import  default_backend
from cryptography.hazmat.primitives.asymmetric import utils
import time
import os
import getpass

# Function to generate an AES key using PBKDF2 with a SHA256 hash function
def generate_aes_key(password, salt, count, key_length):
  return PBKDF2(password, salt, key_length, count=count, hmac_hash_module=SHA256)

# Function to generate RSA key pair for a user
def generate_rsa_key_pair():
    key = RSA.generate(2048)
    # Saving the user 1's public key to a file
    public_key = key.publickey().export_key()
    with open("user1_public_key.pem", "wb") as public_file:
      public_file.write(public_key)

    # Saving the user 2's public key to a file
    public_key = key.publickey().export_key()
    with open("user2_public_key.pem", "wb") as public_file:
      public_file.write(public_key)

    # Saving the user 1's private key to a file
    private_key = key.export_key()
    with open("user1_private_key.pem", "wb") as private_file:
      private_file.write(private_key)

    # Saving the users 2's private key to a file
    private_key = key.export_key()
    with open("user2_private_key.pem", "wb") as private_file:
      private_file.write(private_key)

    # Returning the generated public and private keys
    return private_key, public_key

# Function to encrypt a file using AES-GCM mode
def encrypt_file(filename, user1_public_key, user2_public_key):
  # Asking the user for the first encryption password and confirmation
  while True:
    password1 = getpass.getpass(prompt="Enter the first encryption password: ")
    password2 = getpass.getpass(prompt="Confirm encryption password: ")

    # Checking if the passwords match
    if password1 == password2:
      break
    else:
      print("Passwords do not match. Please try again.")

  # Asking the user for the second encryption password and confirmation
  while True:
    password3 = getpass.getpass(prompt="Enter the second encryption password: ")
    password4 = getpass.getpass(prompt="Confirm encryption password: ")

    # Check if the passwords match
    if password3 == password4:
      break
    else:
      print("Passwords do not match. Please try again.")

  # Generating a random salt for both keys
  salt1 = get_random_bytes(16)
  salt2 = get_random_bytes(16)

  # Generating two AES keys using PBKDF2 with the chosen password and salt
  aes_key1 = generate_aes_key(password1.encode(), salt1, 100000, 32)
  aes_key2 = generate_aes_key(password3.encode(), salt2, 100000, 32)

  # Creating AES cipher objects in GCM mode with generated keys and salts as nonces
  aes_cipher1 = AES.new(aes_key1, AES.MODE_GCM, nonce=salt1)
  aes_cipher2 = AES.new(aes_key2, AES.MODE_GCM, nonce=salt2)

  # Generating RSA key pairs for two users
  private_key_user1, public_key_user1 = generate_rsa_key_pair()
  private_key_user2, public_key_user2 = generate_rsa_key_pair()

  # Encrypting the AES keys with the both users' public keys
  encrypted_aes_key1 = PKCS1_OAEP.new(RSA.import_key(public_key_user1)).encrypt(aes_key1)
  encrypted_aes_key2 = PKCS1_OAEP.new(RSA.import_key(public_key_user2)).encrypt(aes_key2)

  # Set the output filename for the encrypted file
  output_filename = filename + '.enc'

  # Opening the input file in binary read mode and output file in binary write mode
  with open(filename, 'rb') as input_file, open(output_filename, 'wb') as output_file:
    start_time = time.time()

    # Writing the salts and ecnrypted AES keys for User 1 and User 2 to the output file
    output_file.write(salt1)
    output_file.write(salt2)
    output_file.write(encrypted_aes_key1)
    output_file.write(encrypted_aes_key2)

    # Reading and Processing the input file in 1 MB blocks
    while True:
      block = input_file.read(1024 * 1024) # Read 1 MB block

      # Break if no more blocks to process
      if not block:
        break

      # Encrypting the block and obtain the authentication tag
      cipher1, tag1 = aes_cipher1.encrypt_and_digest(pad(block, AES.block_size))
      cipher2, tag2 = aes_cipher2.encrypt_and_digest(pad(block, AES.block_size))

      # Writing the encrypted block and tag and authentication tag for User 1 and User 2 to the output file
      output_file.write(cipher1) # Writes the encrypted block for User 1
      output_file.write(tag1) # Writes the authentication tag for User 1
      output_file.write(cipher2) # Writes the encrypted block for User 2
      output_file.write(tag2) # Writes the authentication tag for User 2

    # Recording the end time after proccessing entire file
    end_time = time.time()

  # Displaying encryption completion information underneath in console
  print("Encryption completed.")
  print("User 1's Private key saved to user1_private_key.pem")
  print("User 1's Public key saved to user1_public_key.pem")
  print("User 2's Public key saved to user2_public_key.pem")
  print("User 2's Private key saved to user2_private_key.pem")
  print("Salt1:", salt1.hex())
  print("Salt2:", salt2.hex())
  print("Time taken:", end_time - start_time, "seconds")
  print("Encrypted file saved as", output_filename)




# Function to decrypt a file encrypted with AES-GCM mode
def decrypt_file(filename, user_private_key):
  # Geting the decryption password from the user without showing it on the screen
  password1 = getpass.getpass(prompt="Enter the first decryption password: ")
  password2 = getpass.getpass(prompt="Enter the first decryption password: ")

  # Opening the encrypted file in binary mode (Specified by 'filename')
  with open(filename, 'rb') as input_file:
    # Reading the salt from the beginning of the file
    salt1 = input_file.read(16) # Reads the frist 16 bytes for the salt of AES key 1
    salt2 = input_file.read(16) # Reads the next 16 bytes for the salt of AES key 2
    # Reads the encrypted AES keys from the file
    encrypted_aes_key1 = input_file.read(256) # Reads 256 bytes for the encrypted AES key 1
    encrypted_aes_key2 = input_file.read(256) # Reads 256 bytes for the encrypted AES key 2

    # Decrypting the RSA private key to obtain AES keys
    # Decrypting the RSA private key for user 1 to obtain the corresponding AES key
    # Uses the password, salt, iteration count(100000), and key length(32) in the key derivation process
    aes_key1 = generate_aes_key(password1.encode(), salt1, 100000, 32)
    # Decrypting the RSA private key for user 1 to obtain the corresponding AES key
    # Uses the password, salt, iteration count(100000), and key length(32) in the key derivation process
    aes_key2 = generate_aes_key(password2.encode(), salt2, 100000, 32)

    # Creating an AES cipher objects in GCM mode with the specified nonces (salts)
    # (Galois/Counter Mode) MODE_GCM used for authenticated encryption
    # Creating an AES cipher object for the User 1 with the generated AES key and salt
    # The nonce is set to the first salt
    aes_cipher1 = AES.new(aes_key1, AES.MODE_GCM, nonce=salt1)
    # Creating an AES cipher object for the User 2 with the generated AES key and salt
    # The nonce is set to the second salt
    aes_cipher2 = AES.new(aes_key2, AES.MODE_GCM, nonce=salt2)

    # Setting the output filename for the decrypted file
    output_filename = filename[:-4] + '.dec'

    # Opening the output file in binary write mode
    with open(output_filename, 'wb') as output_file:
      while True:
        # Reading and processing the encrypted file in 1 MB blocks
        block1 = input_file.read(1024 * 1024)
        block2 = input_file.read(1024 * 1024)
        if not block1 or block2:
          break

        # Separating the cipher and tag from the block
        cipher1 = block1[:-16] # Cipher without the tag
        tag1 = block1[-16:] # The last 16 bytes represent the tag

        cipher2 = block2[:-16] # Cipher without the tag
        tag2 = block2[-16:] # The last 16 bytes represent the tag

        # Decrypting and verifying the first and second block using the cipher and tag
        decrypted1 = aes_cipher1.decrypt_and_verify(cipher1, tag1)
        decrypted2 = aes_cipher2.decrypt_and_verify(cipher2, tag2)

        # Removing the padding and writing the decrypted block to the output file
        # Wrting the decypted first and second block without padding to the output file
        output_file.write(unpad(decrypted1, AES.block_size))
        output_file.write(unpad(decrypted2, AES.block_size))

  # Displays the decryption completion underneath in console
  print("Decryption completed.")
  print("Decrypted file saved as", output_filename)


# Main function
def main():
  # Encrypting the files for User 1 and User 2
  filename = "user1encryptionfile.txt" # The filename I created
  # Public keys for the two users involved in the encryption
  user1_public_key = "public_key_user1"
  user2_public_key = "public_key_user2"
  # Public keys for the two users involved in the encryption
  encrypt_file(filename, user1_public_key, user2_public_key)

  # Setting the filename for the second user's encryption file
  filename = "user2encryptionfile.txt"
  # Public keys for the two users involved in the encryption
  user1_public_key = "public_key_user1"
  user2_public_key = "public_key_user2"
  # Calling the encrypt_file function with the specified parameters
  encrypt_file(filename, user1_public_key, user2_public_key)

  # Decrypting the files for User 1 and User 2
  # The file containing the private key for User 1
  filename = "user1encryptionfile.txt.enc" # The encrypted version of file, Could not save to decryptionfile.txt.dec.
  # The file that contains the private key for User 1
  user1_private_key_file = "user1_private_key.pem"
  # Imports the RSA private key for User 1 from 'user1_private_key_file'
  user_private_key = RSA.import_key(open(user1_private_key_file).read())
  # Calls the 'decrypt_file' function passing the file name of the encrypted file and the private key for User 1 as parameters
  decrypt_file(filename, user_private_key)

  # This line sepcifies that the filename of the encrypted file needs to be encrypted.
  filename = "user2encryptionfile.txt.enc" # The encrypted version of file, Could not save to decryptionfile.txt.dec
  # The file containing the private key for user 2
  user2_private_key_file = "user2_private_key.pem"
  # Opens the 'user_private_key_file' containing the private key for user 2, read its content and imports it as an RSA private key.
  user_private_key = RSA.import_key(open(user2_private_key_file).read())
  # Calling the 'decrypt_file' function passsing the filename of the encrypted file and user's private key as parameters.
  # This function is responsible for decrypting the specified file using AES-GCM with the provided private key.
  decrypt_file(filename, user_private_key)

# Entry point for the script, Checks if the script is executed and not imported as a module.
# Calls the main function to execute the ecryption and decryption processes
if __name__ == "__main__":
  main()


Enter the first encryption password: ··········
Confirm encryption password: ··········
Enter the second encryption password: ··········
Confirm encryption password: ··········
Encryption completed.
User 1's Private key saved to user1_private_key.pem
User 1's Public key saved to user1_public_key.pem
User 2's Public key saved to user2_public_key.pem
User 2's Private key saved to user2_private_key.pem
Salt1: 70e1a566b3346764a72851083c855c8e
Salt2: fcc419893162406bdb020e293f1b36e5
Time taken: 1.1920928955078125e-05 seconds
Encrypted file saved as user1encryptionfile.txt.enc
Enter the first encryption password: ··········
Confirm encryption password: ··········
Enter the second encryption password: ··········
Confirm encryption password: ··········
Encryption completed.
User 1's Private key saved to user1_private_key.pem
User 1's Public key saved to user1_public_key.pem
User 2's Public key saved to user2_public_key.pem
User 2's Private key saved to user2_private_key.pem
Salt1: f2302b049729

In [ ]:
# Importing necessary modules from the Crypto library
from Crypto.Signature import pkcs1_15
from Crypto.Hash import SHA256
from Crypto.PublicKey import RSA
import getpass

# Function to generate RSA key pair for a user
def generate_rsa_key_pair():
    key = RSA.generate(2048)
    # Saving the user 1's public key to a file
    public_key_filename = "user1_public_key.pem"
    with open(public_key_filename, "wb") as public_file:
        public_file.write(key.publickey().export_key())

    # Saving the user 2's public key to a file
    public_key_filename = "user2_public_key.pem"
    with open(public_key_filename, "wb") as public_file:
        public_file.write(key.publickey().export_key())

    # Saving the user 1's private key to a file
    private_key_filename = "user1_private_key.pem"
    with open(private_key_filename, "wb") as private_file:
        private_file.write(key.export_key())

    # Saving the user 2's private key to a file
    private_key_filename = "user2_private_key.pem"
    with open(private_key_filename, "wb") as private_file:
        private_file.write(key.export_key())

    return private_key_filename, public_key_filename

# Function to sign a message using RSA private key
def sign_message(private_key, message):
    # Importing the private key
    key = RSA.import_key(private_key)
    # Created a SHA-256 hash object from the message
    h = SHA256.new(message.encode())
    # Signing the hash using PKCS#1.5 padding
    signature = pkcs1_15.new(key).sign(h)
    return signature

# Function that verify the signature of a message using RSA public key
def verify_signature(public_key, message, signature):
    # Importing the public key
    key = RSA.import_key(public_key)
    # Created a SHA-256 hash object from the message
    h = SHA256.new(message.encode())
    try:
        # Verifying the signature using PKCS#1.5 padding
        pkcs1_15.new(key).verify(h, signature)
        return True  # Signature is valid
    except (ValueError, TypeError):
        return False  # Signature is not valid

# Example usage
def main():
    # RSA keys for user 1
    user1_private_key = open("user1_private_key.pem").read()
    user1_public_key = open("user1_public_key.pem").read()

    # RSA keys for user 2
    user2_private_key = open("user2_private_key.pem").read()
    user2_public_key = open("user2_public_key.pem").read()

    encrypted_filename = "encryptionfile.txt.enc"

    # Geting the decryption passwords from the user without showing them on the screen
    password1 = getpass.getpass(prompt="Enter the first decryption password: ")
    password2 = getpass.getpass(prompt="Enter the second decryption password: ")

    # Combine the passwords to form a message
    message = password1 + password2

    # Signing the message using user 1's private key
    signature_user1 = sign_message(user1_private_key, message)
    print("User 1's Signature:", signature_user1.hex())

    # Signing the message using user 2's private key
    signature_user2 = sign_message(user2_private_key, message)
    print("User 2's Signature:", signature_user2.hex())

    # Verifying the signatures using user 1's public key
    is_valid_user1 = verify_signature(user1_public_key, message, signature_user1)
    print("Is User 1's signature valid?", is_valid_user1)

    # Verifying the signatures using users 2's public key
    is_valid_user2 = verify_signature(user2_public_key, message, signature_user2)
    print("Is User 2's signature valid?", is_valid_user2)

    # Decrypt the file using your existing decryption function
    #recipient1_private_key = RSA.import_key(recipient1_private_key)
    #decrypt_file(encrypted_filename, recipient1_private_key)

# Entry point for the script
if __name__ == "__main__":
    main()



Enter the first decryption password: ··········
Enter the second decryption password: ··········
User 1's Signature: 2d9a4370479a84e4320bc94e590b8b48a58f1710c384d3b0f50ff249599b3f7f9763fa90dfda688081457f59b153d76777801796d894053862a6212f8c9be53b1cdd08d13683840195d226acef811eb74341336f7ff71e5364ef7caa8d2fa93dff8e233499729a9649391e3e252234028c332a0b426f5e188705051902246fac675eb1939ae18b244261c8a27c9cff51c0b881bca56e74e9b511839ed5206d0a710fcb0d97f10ce83b4bc4be79d3227a8b8b3e973cc9fa5ad4609fd98315f7d60b8c1532cc1fcec8d1e64d3fe38d501585885ddcd1d1b1cbeac35e3597e6a961a97180e245cccd1fec676d77fbb87e9e98ba059a8915abf3e5c4ac8da3eefdcc
User 2's Signature: 2d9a4370479a84e4320bc94e590b8b48a58f1710c384d3b0f50ff249599b3f7f9763fa90dfda688081457f59b153d76777801796d894053862a6212f8c9be53b1cdd08d13683840195d226acef811eb74341336f7ff71e5364ef7caa8d2fa93dff8e233499729a9649391e3e252234028c332a0b426f5e188705051902246fac675eb1939ae18b244261c8a27c9cff51c0b881bca56e74e9b511839ed5206d0a710fcb0d97f10ce83b4bc4be79d322